In [10]:
# -*- coding: utf-8 -*-
import requests
import re
from bs4 import BeautifulSoup as BS
import pandas as pd
import json
from collections import Counter
from urllib.error import HTTPError
import numpy as np
from tqdm import trange

In [11]:
def regex(name):
    if name == "和田地区和田县" or name == "和田地区和田市":
        return name[-3:]
    
    name = name.strip()
    name = name.strip('\t')
    special = ['城区','矿区','郊区'] ## with which patterns can only use exact match
    for item in special:
        if item in name:
            return name
    
    with open('./ethicity.json','r') as f:
        ethnicities = json.loads(f.read())
    ethnicity = [item["name"] for item in ethnicities] + ["各族"]

    for eth in ethnicity:
        if eth in name:
            name = name.replace(eth,'')
            
    if '地区' in name:
        name = [item for item in name.split("地区") if len(item)>0][-1]
    if '市' in name:
        name = [item for item in name.split("市") if len(item)>0][-1]
    elif '自治州' in name:
        name = [item for item in name.split("自治州") if len(item)>0][-1]
    
    if len(name) > 2:
        name = name.strip('区')
        name = name.strip('旗')
        name = name.strip('自治县')
        name = name.strip('省')
    
    if len(name) > 2:
        name = name.strip('县')
        
    return name

In [12]:
All_data = pd.read_excel('./Alldata.xlsx',keep_default_na=False).drop(columns=['Unnamed: 0'],axis=1)
indices = []
for i in range(len(All_data)):
    line = All_data.iloc[i]
    indices.append((line['t'],regex(line['area'])))
All_data['index'] = indices
All_data['gov_report'] = ''
All_data['stat_report'] = ''

In [13]:
## crawl data in areas listed in indices
def craw_and_write(web_path_root, to_be_matched_list = indices):
    global All_data
    if "tongjigongbao" in web_path_root:
        entry = 'stat_report'
    else:
        entry = 'gov_report'
    count_dict = {key:0 for key in to_be_matched_list}
    
    for i in trange(25000):
        title, body = craw_single(f'{web_path_root}/{i}.html')
        ## try matching
        if title:
            for k in range(len(to_be_matched_list)): ## item: (t, area)
                item = to_be_matched_list[k]
                if (str(item[0]) in title) and (item[1] in title): ## if matched
                    print(item)
                    All_data.loc[k,entry] = (title+'\t'+body) ## data: (title, body)
                    count_dict[item] += 1
                    break
        ## whether matched successfully, go to next
        i += 1
    return count_dict


In [14]:
def craw_single(web_path):
    r = requests.get(web_path).text.encode('ISO-8859-1').decode('gbk')
    title = re.findall(r'<div class="title"><h1>(.*?)</h1></div>',r)
    if len(title) > 0: ## if not "404 Not Found" returned
        title = title[0]
        d = pd.read_html(web_path,flavor='bs4')
        body = d[0][0].values[np.argmax([len(item) for item in d[0][0].values])]
        j = 2
        while True:
            try:
                e = pd.read_html(web_path[:-5]+f'_{j}'+'.html',flavor='bs4')
                if len(e)>0:
                    data_ = e[0][0].values[np.argmax([len(item) for item in e[0][0].values])]
                    body = body + data_
                j += 1
            except HTTPError:
                return (title, body)
    else:
        return None, None
            
    

In [15]:
## main ##
count_dict = craw_and_write('https://www.ahmhxc.com/gongzuobaogao/')
All_data.to_excel('./WITH_GOV_REPORT.xlsx')
with open('./gov_report.json','w') as g:
    json.dump(count_dict)

KeyboardInterrupt: 

['（新疆）和田县2017年国民经济和社会发展统计公报',
 '和田县2017年国民经济和社会发展统计  公 报  2017年，全县各部门在以习近平同志为核心的党中央坚强领导下，不断增强“四个意识”，深入贯彻落实党的十八大和十八届三中、四中、五中、六中、七中全会精神，认真学习贯彻党的十九大精神，以习近平新时代中国特色社会主义思想为指导。在自治区党委、人民政府的亲切关怀和地委行署的坚强领导下，在北京市的大力援助下，和田县委、人民政府团结带领全县各族干部群众，深入学习贯彻习近平总书记系列重要讲话精神，坚持稳中求进工作总基调，坚定不移贯彻新发展理念，坚持以提高发展质量和效益为中心，统筹推进稳增长、促改革、调结构、惠民生、防风险各项工作，围绕总目标、适应新常态，科学分析和田县2017年经济发展现状，完善发展思路，经济运行稳中有进、稳中向好、好于预期，经济社会保持平稳健康发展。  一、综 合  初步核算，2017年和田县实现生产总值(GDP)342469万元，同比增长9.2%。分三次产业来看，第一产业完成86290万元，同比增长2.7%;第二产业完成83217万元，同比增长7.4%;第三产业完成172962万元，同比增长16.1%。三次产业占生产总值的比重分别为25.2%、24.3%、50.5%。按户籍年平均常住人口数计算，人均生产总值9657元。 二、农林牧渔业  2017年度全县农林林渔服务业(现价)总产值达233678.5万元，其中：种植业总产值170225.9万元，林业总产值1797.87万元，畜牧业总产值58633.47万元，渔业总产值1412.25万元。现价增加值达98487.84万元。  农林林渔服务业(可比价)总产值达238915.8万元，种植业总产值176106.6万元，林业总产值1797.87万元，畜牧业总产值57997.19万元，渔业总产值1391.9万元。可比价增加值达100076.7万元，可比价增加值发展速度2.6%。  ——种植业生产  (一)粮食：2017年粮食面积41.42万亩，同比增长0.84%，粮食产量15.98万吨，同比增长3.64%，单产386公斤，同比增长0.64%;其中：夏粮面积24.4万亩，同比增长2.5%，夏粮产量达9.18万吨，同比增长2.9%，粮食单产376公斤，同比增长0.4%;玉米面积13.42万亩，同比下降1.46%，

In [ ]:
All_data.index

Index([(2013, '卢氏'), (2014, '卢氏'), (2015, '卢氏'), (2016, '卢氏'), (2017, '卢氏'),
       (2018, '卢氏'), (2019, '卢氏'), (2013, '上饶'), (2014, '上饶'), (2015, '上饶'),
       ...
       (2017, '龙江'), (2018, '龙江'), (2019, '龙江'), (2013, '龙井'), (2014, '龙井'),
       (2015, '龙井'), (2016, '龙井'), (2017, '龙井'), (2018, '龙井'), (2019, '龙井')],
      dtype='object', name='index', length=5824)

In [ ]:
d[0][0].values[np.argmax([len(item) for item in d[0][0].values])]

'赤峰市2020年  国民经济和社会发展统计公报[1]  赤峰市统计局  (2021年3月26日)  2020年，面对严峻复杂的经济形势、艰巨繁重的改革发展稳定任务，特别是新冠肺炎疫情的严重冲击，市委、市政府坚持以习近平新时代中国特色社会主义思想为指导，全面贯彻党的十九大和十九届二中、三中、四中、五中全会精神，按照自治区党委、政府工作部署，扎实做好“六稳”工作，全面落实“六保”任务，统筹推进常态化疫情防控和经济社会发展取得显著成效，全市经济实现回稳向好，社会大局保持和谐稳定，为“十四五”良好开局打下坚实基础。  一、综合  初步核算，全年地区生产总值[2]1763.6亿元，按可比价计算，比上年增长1.4%。其中，第一产业增加值346.4亿元，增长0.8%;第二产业增加值550亿元，增长6.4%;第三产业增加值867.2亿元，下降1.4%。三次产业比例为19.6:31.2:49.2。  全年城镇新增就业人数2.4万人，比上年减少0.2万人。失业人员再就业1.1万人，农村牧区劳动力转移就业人口69.5万人次。年末城镇登记失业率4.08%，比上年提高0.01个百分点。  全年居民消费价格比上年上涨2%。其中，消费品价格上涨3.1%，服务价格下降0.3%。商品零售价格上涨1.3%，工业生产者出厂价格下降0.3%。  表12020年居民消费价格比上年涨跌幅度  指 标  比上年增长（%）  居民消费价格  2.0  其中：食品烟酒  6.7  其中：粮食  -0.1  鲜菜  10.4  畜肉  28.1  水产品  -7.1  蛋  -18.1  鲜瓜果  -1.8  衣着  0.9  居住[3]  -0.7  生活用品及服务  0.0  交通和通信  -1.9  教育文化和娱乐  0.6  医疗保健  0.1  其他用品和服务  1.9  二、农牧业  全年农作物总播种面积142.2万公顷，比上年增长0.3%。其中，粮食作物播种面积111.2万公顷，增长0.7%;经济作物播种面积31万公顷，下降1%。粮食总产量611.7万吨，比上年增长1.1%。经济作物中，油料产量17.3万吨，下降15.6%;甜菜产量197.9万吨，增长10.5%;蔬菜及食用菌产量324万吨，下降2.1%;水果产量34.5万吨，下降0.5%。'

In [ ]:
'''
政府工作报告中可以查到：
1.城镇居民人均可支配收入
2.农村居民人均可支配收入
3.地区生产总值
4.固定资产投资
5.社会销售品零售总额

国民经济和社会发展统计公报
1.年末总人口（户籍）
2.财政支出
3.普通小学在校生
4. 医院、卫生院床位数
5.农村用电量
'''

In [ ]:
len(d.split(' ')[1])

0

In [ ]:
d = pd.read_html('https://www.ahmhxc.com/tongjigongbao/21204.html',flavor='bs4')[0][0][0]


In [ ]:
d.split(' ')

['湘潭县2020年国民经济和社会发展统计公报',
 '',
 '2020年是极不平凡的一年。面对国内外形势的深刻复杂变化和新冠肺炎疫情的冲击，全县上下在县委、县政府的坚强领导下，坚持以习近平新时代中国特色社会主义思想为指导，认真贯彻习近平总书记考察湖南重要讲话指示精神，全面落实党中央、国务院、省委、省政府和市委、市政府各项决策部署，坚持稳中求进工作总基调，统筹疫情防控和经济社会发展，扎实做好“六稳”工作，全面落实“六保”任务，继续实施“强工惠农”发展战略，开展“五大竞赛”，着力推动县域高质量发展。2020年，全县经济运行呈增速稳步回升、结构持续优化、质效不断改善的态势。',
 '',
 '一、综合',
 '',
 '初步核算，全年实现地区生产总值501.0亿元，增长3.7%。其中，第一产业增加值68.6亿元，增长3.7%;第二产业增加值268.9亿元，增长4.4%;第三产业增加值163.5亿元，增长2.2%。三次产业结构比为13.7:53.7:32.6。',
 '',
 '二、农业',
 '',
 '全年实现农林牧渔业总产值119.5亿元，增长3.8%。全县粮食播种面积83.6千公顷，下降2.9%，粮食总产量60.65万吨，下降2.7%;蔬菜播种面积20.6千公顷，增长3.5%，蔬菜产量64.41万吨，增长4.5%;油料播种面积13.1千公顷，增长18.4%，油料产量2.2万吨，增长18.4%。全年猪牛羊禽肉产量7.2万吨，下降3.9%。其中，猪肉产量7万吨，下降4.3%。禽蛋产量3.0万吨，增长20.3%。水产品产量4.3万吨，增长12.1%。年末出栏生猪96.9万头，下降3.8%;出栏牛1.2万头，增长35.3%;出栏羊6.5万头，增长20.6%。',
 '',
 '全年水利工程投入资金4.6亿元，开工各类水利工程7500处，完成水利工程土方石230万立方米。新增农田有效灌溉面积500公顷，新增节水灌溉面积660公顷。全县年末农业机械总动力达147.1万千瓦。',
 '',
 '三、工业和建筑业',
 '',
 '全年完成规模工业增加值203.4亿元，增长3.4%，其中天易示范区完成规模工业增加值166.7亿元，增长3.8%，占全县规模工业增加值的比重为82.0%。全县规模工业企业实现主营业务收入621.8亿元，下降1.3%，实现利润总额22.8亿元，增长0.